# REST-APIs mit FastAPI
**Für Selbststudium und Vertiefung**  
---
### Ziel:
- Theoretisches Verständnis von REST-APIs und FastAPI aufbauen
- Praktische Implementierung aller CRUD-Operationen
- Vertiefung von Datenvalidierung und Fehlerbehandlung
- Einsatz von Testtools wie Postman

## 1. Grundlagen von REST-APIs
### 1.1 Was ist eine API?
Eine **Application Programming Interface (API)** ist ein Vertrag zwischen Systemen:
- **Analogie:** Speisekarte in einem Restaurant  
  - Gäste (Client) bestellen Gerichte (Daten) über die Karte (API-Schnittstelle)  
  - Küche (Server) verarbeitet die Bestellung und liefert das Ergebnis

**Einsatzgebiete:**
- Microservices-Architekturen
- Mobile Apps ↔ Backend-Kommunikation
- Datenaggregation (z.B. Wetterdienste)

### 1.2 REST-Prinzipien
REST ist **kein Standard**, sondern ein Architekturstil mit sechs Constraints:
1. **Client-Server-Trennung**  
   - Client verwaltet UI/User-State, Server verwaltet Daten
2. **Zustandslosigkeit (Stateless)**  
   - Jeder Request enthält **alle notwendigen Informationen**  
   - Beispiel: Authentifizierungstoken muss in jedem Request mitgesendet werden
3. **Caching**  
   - Responses können zwischengespeichert werden (z.B. mit `Cache-Control`-Header)
4. **Einheitliche Schnittstelle**  
   - Ressourcenidentifikation über URLs (z.B. `/items/42`)
   - Selbstbeschreibende Messages (JSON/XML)
5. **Layered System**  
   - Client kennt nur die API, nicht die Server-Infrastruktur dahinter
6. **Code-on-Demand (optional)**  
   - Server kann ausführbaren Code senden (z.B. JavaScript)

### 1.3 HTTP-Methoden im Detail
| Methode | Idempotent? | Sicher? | Verwendung |
|---------|-------------|---------|------------|
| GET     | Ja          | Ja      | Daten abrufen |
| POST    | Nein        | Nein    | Ressource erstellen |
| PUT     | Ja          | Nein    | Ressource vollständig ersetzen |
| PATCH   | Nein        | Nein    | Ressource teilweise aktualisieren |
| DELETE  | Ja          | Nein    | Ressource löschen |

**Idempotenz-Beispiel:**  
- `DELETE /items/42` → Auch bei mehrfacher Ausführung bleibt das Ergebnis gleich (Item 42 gelöscht)
- `POST /items` → Jede Ausführung erstellt ein neues Item (nicht idempotent)

## 2. FastAPI-Architektur
### 2.1 ASGI vs. WSGI
| Feature          | WSGI (Flask/Django) | ASGI (FastAPI/Starlette) |
|------------------|----------------------|--------------------------|
| **Verarbeitung** | Synchron            | Asynchron               |
| **Performance**  | Gering bei Blocking I/O | Hoch durch Async/Await |
| **Use-Case**     | Traditionelle Web-Apps | Echtzeit-APIs, WebSockets |

**Beispiel für Async-Endpoint:**
```python
@app.get("/slow-endpoint")
async def slow_operation():
    await asyncio.sleep(5)  # Non-blocking Wartezeit
    return {"status": "done"}
```

### 2.2 Dependency Injection
FastAPI verwaltet Abhängigkeiten automatisch:
```python
from fastapi import Depends

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@app.get("/users")
def read_users(db: Session = Depends(get_db)):
    return db.query(User).all()
```
- `Depends()` injiziert die Datenbankverbindung in den Endpoint
- Automatisches Öffnen/Schließen der Verbindung

## 3. Datenvalidierung mit Pydantic
### 3.1 Erweiterte Feldvalidierung
```python
from pydantic import BaseModel, Field, confloat

class Item(BaseModel):
    name: str = Field(..., min_length=2, max_length=50)
    price: confloat(gt=0)  # Preis muss > 0 sein
    category: str = Field(regex="^(tools|consumables)$")

    # Beispiel für benutzerdefinierte Validierung
    @validator('name')
    def name_must_contain_letter(cls, v):
        if 'a' not in v.lower():
            raise ValueError('Name muss den Buchstaben "a" enthalten')
        return v
```

**Fehlerantwort bei Validierungsfehler:**
```json
{
  "detail": [
    {
      "loc": ["body", "price"],
      "msg": "ensure this value is greater than 0",
      "type": "value_error.number.not_gt"
    }
  ]
}
```

### 3.2 Response-Modelle
```python
from typing import List

class ItemResponse(BaseModel):
    id: int
    name: str

@app.get("/items", response_model=List[ItemResponse])
def get_items():
    return items.values()  # Automatische Filterung auf response_model-Felder
```

## 4. CRUD-Implementierung Schritt-für-Schritt
### 4.1 Datenstruktur und Initialisierung
```python
from uuid import UUID, uuid4
from datetime import datetime

class AdvancedItem(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    created_at: datetime = Field(default_factory=datetime.now)
    name: str
    price: float

items: dict[UUID, AdvancedItem] = {}
```
- **UUIDs** vermeiden ID-Konflikte in verteilten Systemen
- **created_at** zeigt Erstellungszeitpunkt

### 4.2 POST mit Statuscodes
```python
from fastapi import status

@app.post(
    "/items",
    status_code=status.HTTP_201_CREATED,
    response_model=AdvancedItem
)
def create_item(item: AdvancedItem):
    items[item.id] = item
    return item
```
- **Statuscode 201** statt 200 für klare Semantik
- **response_model** filtert Rückgabefelder

## 5. Testen mit Postman
### 5.1 Collection-Import
1. **Neue Collection erstellen** → "FastAPI Demo"
2. **Environment-Variablen definieren:**  
   - `base_url`: `http://localhost:8000`
3. **Beispiel-Request:**  
   - **GET:** `{{base_url}}/items`  
   - **Tests:**  
     ```javascript
     pm.test("Status 200", () => pm.response.to.have.status(200));
     pm.test("JSON Body", () => {
         const json = pm.response.json();
         pm.expect(json).to.be.an("array");
     });
     ```

### 5.2 Automatisierte Tests
**Beispiel mit pytest:**
```python
from fastapi.testclient import TestClient
client = TestClient(app)

def test_create_item():
    response = client.post("/items", json={"name": "Test", "price": 9.99})
    assert response.status_code == 201
    assert isinstance(response.json()["id"], str)
```

## 6. Weiterführende Konzepte
### 6.1 Middlewares
```python
from fastapi import Request

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    response.headers["X-Process-Time"] = str(time.time() - start_time)
    return response
```
- **Anwendungsfälle:** Logging, CORS, Rate-Limiting

### 6.2 Hintergrundtasks
```python
from fastapi import BackgroundTasks

def log_operation(item_id: UUID):
    with open("log.txt", "a") as f:
        f.write(f"Processed item {item_id}\n")

@app.post("/items")
def create_item(
    item: AdvancedItem,
    background_tasks: BackgroundTasks
):
    background_tasks.add_task(log_operation, item.id)
    return item
```
- **Asynchrone Verarbeitung** ohne Blockieren des Hauptthreads

## 7. Übungen & Lösungsansätze
### Aufgabe 1: PATCH-Endpoint
**Ziel:** Teilaktualisierung mit `PATCH /items/{id}`  
**Lösung:**
```python
from fastapi import Body

@app.patch("/items/{item_id}")
def update_item(
    item_id: UUID,
    name: str = Body(None),
    price: float = Body(None)
):
    item = items[item_id]
    if name:
        item.name = name
    if price:
        item.price = price
    return item
```

### Aufgabe 2: Paginierung
**Ziel:** `GET /items?skip=0&limit=10`  
**Lösung:**
```python
from fastapi import Query

@app.get("/items")
def get_items(
    skip: int = Query(0, ge=0),
    limit: int = Query(10, le=100)
):
    return list(items.values())[skip : skip + limit]
```

## 8. Weiterführende Ressourcen
- **Offizielle Dokumentation:** [fastapi.tiangolo.com](https://fastapi.tiangolo.com/)
- **Buch:** "Building Data Science Applications with FastAPI" (Packt)
- **Best Practices:** [fastapi-best-practices](https://github.com/zhanymkanov/fastapi-best-practices)
- **Security:** OAuth2 mit [FastAPI Users](https://github.com/fastapi-users/fastapi-users)

**Abschlussaufgabe:**  
Erstelle eine vollständige API für ein Bibliothekssystem mit:
- Büchern (Titel, Autor, ISBN)
- Ausleihen (Benutzer-ID, Buch-ID, Fälligkeitsdatum)
- Suchfilter (Autor, Veröffentlichungsjahr)
- Automatischen Overdue-Benachrichtigungen per Background-Task